# Traffic Accidents in Brazillian Federal Highways

In [6]:
import mysql.connector as mysql

In [9]:
mydb = mysql.connect(
  host="localhost",
  user="root",
  password="Vini_2002"
)

In [10]:
cursor = mydb.cursor()

In [13]:
cursor.execute("CREATE DATABASE brazil_traffic;")

In [ ]:
cursor.execute("""
CREATE TABLE Ocorrencias (
    
)
""")